In [1]:
%pwd

'/home/hajibagher/insect-classification-deeplearning'

In [23]:
import torch
from torch import nn
import torchviz

import vit
import scipy.io
from matplotlib import pyplot as plt
import numpy as np
import dnaencoder
import pandas as pd
from importlib import reload

In [24]:
from mat import mat

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
print("Device", device)

Device cuda


In [47]:
dict(dnaencoder.evaluate_model(mat, "amintehrani/dnaencoder-insects-finetuned"))

Evaluating model: amintehrani/dnaencoder-insects-finetuned
Number of classes: 1051
Validation samples: 3234
Evaluating validation dataset...


Evaluating validation dataset...


Starting test evaluation...


Starting test unseen evaluation...


{'val_seen': {'eval_loss': 3.958054304122925,
  'eval_model_preparation_time': 0.0052,
  'eval_accuracy': 0.7878787878787878,
  'eval_runtime': 245.3919,
  'eval_samples_per_second': 13.179,
  'eval_steps_per_second': 0.416},
 'val_unseen': {'eval_loss': 7.709295749664307,
  'eval_model_preparation_time': 0.0056,
  'eval_accuracy': 0.0,
  'eval_runtime': 282.3414,
  'eval_samples_per_second': 13.179,
  'eval_steps_per_second': 0.414},
 'test_seen': {'eval_loss': 4.644564151763916,
  'eval_model_preparation_time': 0.0054,
  'eval_accuracy': 0.643687374749499,
  'eval_runtime': 378.7899,
  'eval_samples_per_second': 13.174,
  'eval_steps_per_second': 0.412},
 'test_unseen': {'eval_loss': 7.724428653717041,
  'eval_model_preparation_time': 0.0053,
  'eval_accuracy': 0.0,
  'eval_runtime': 564.6694,
  'eval_samples_per_second': 13.176,
  'eval_steps_per_second': 0.413}}

In [ ]:
model_name = "amintehrani/vit-insects-finetuned7"
dict(vit.evaluate_model(mat, model_name, device))

[('val_seen_indices',
  {'eval_loss': 0.82353675365448,
   'eval_model_preparation_time': 0.0026,
   'eval_accuracy': 0.8528138528138528,
   'eval_runtime': 16.6445,
   'eval_samples_per_second': 194.299,
   'eval_steps_per_second': 6.128}),
 ('val_unseen_indices',
  {'eval_loss': 10.643092155456543,
   'eval_model_preparation_time': 0.003,
   'eval_accuracy': 0.013705993012631014,
   'eval_runtime': 18.0735,
   'eval_samples_per_second': 205.882,
   'eval_steps_per_second': 6.474}),
 ('test_seen_indices',
  {'eval_loss': 2.636476755142212,
   'eval_model_preparation_time': 0.0029,
   'eval_accuracy': 0.6951903807615231,
   'eval_runtime': 24.294,
   'eval_samples_per_second': 205.401,
   'eval_steps_per_second': 6.421}),
 ('test_unseen_indices',
  {'eval_loss': 10.732027053833008,
   'eval_model_preparation_time': 0.0029,
   'eval_accuracy': 0.01599462365591398,
   'eval_runtime': 36.441,
   'eval_samples_per_second': 204.166,
   'eval_steps_per_second': 6.394})]

In [7]:
all_images = mat['all_images']
all_labels = mat['all_labels'].squeeze()
train_indices = mat['train_loc'].squeeze()
val_seen_indices = mat['val_seen_loc'].squeeze()
val_unseen_indices = mat['val_unseen_loc'].squeeze()
test_seen_indices = mat['test_seen_loc'].squeeze()
test_unseen_indices = mat['test_unseen_loc'].squeeze()

val_indices = np.concatenate((val_seen_indices, val_unseen_indices))
test_indices = np.concatenate((test_seen_indices, test_unseen_indices))

total_images = all_images.shape[0]
print(train_indices.shape[0], val_seen_indices.shape[0], val_unseen_indices.shape[0], test_seen_indices.shape[0], test_unseen_indices.shape[0])



13039 3234 3721 4990 7440


In [52]:
mat["val_seen_loc"].shape

(1, 3234)

In [53]:
from multimodal_dataset import MultiModalDataset
from load_embeddings import load_dna_embeddings, load_img_embeddings

all_dna_features = load_dna_embeddings()
all_image_features = load_img_embeddings()

all_dna_len = list(map(lambda s: len(s.strip()), mat['all_string_dnas']))
dna_str_len_mapping: dict[int,int] = {}

def dna_str_len_to_int(s_len):
    if s_len not in dna_str_len_mapping:
        dna_str_len_mapping[s_len] = len(dna_str_len_mapping)
    return dna_str_len_mapping[s_len]

# def all_dna_len_token():
#     return list(map(dna_str_len_to_int, all_dna_len))

all_dna_len_tokens = list(map(dna_str_len_to_int, all_dna_len))
all_dna_len_tokens = np.array(all_dna_len_tokens, dtype=np.int64)

species2genus = mat['species2genus']-1


genus_species = dict()
max_specie_in_genus = 0
for genus_id, genus in pd.DataFrame(species2genus, columns=['genus']).groupby('genus'):
    specie_indices = genus.index.tolist()
    genus_species[genus_id] = specie_indices
    if len(specie_indices) > max_specie_in_genus:
        max_specie_in_genus = len(specie_indices)

datasets = {
    indicies_type:MultiModalDataset(
        mat['all_string_dnas'][(inds:=mat[indicies_type].flatten())],
        mat['all_images'][inds],
        np.transpose(mat['all_labels'], (1,0))[inds],
        dna_str_len_mapping,
        species2genus,
        genus_species,
        None,
        None,
        dna_embeddings=all_dna_features[inds],
        img_embeddings=all_image_features[inds]
    )
    for indicies_type in ["val_seen_loc", "val_unseen_loc", "test_seen_loc", "test_unseen_loc"]
}
datasets

Loading file: batch_dna_0.pkl
Loading file: batch_dna_1.pkl
Loading file: batch_dna_2.pkl
Loading file: batch_dna_3.pkl
Loading file: batch_dna_4.pkl
Loading file: batch_dna_5.pkl
Loading file: batch_dna_6.pkl
Loading file: batch_dna_7.pkl
Loading file: batch_dna_8.pkl
Loading file: batch_dna_9.pkl
Loading file: batch_dna_10.pkl
Loading file: batch_dna_11.pkl
Loading file: batch_dna_12.pkl
Loading file: batch_dna_13.pkl
Loading file: batch_dna_14.pkl
Loading file: batch_dna_15.pkl
Loading file: batch_dna_16.pkl
Loading file: batch_dna_17.pkl
Loading file: batch_dna_18.pkl
Loading file: batch_dna_19.pkl
Loading file: batch_dna_20.pkl
Loading file: batch_dna_21.pkl
Loading file: batch_dna_22.pkl
Loading file: batch_dna_23.pkl
Loading file: batch_dna_24.pkl
Loading file: batch_dna_25.pkl
Loading file: batch_dna_26.pkl
Loading file: batch_dna_27.pkl
Loading file: batch_dna_28.pkl
Loading file: batch_dna_29.pkl
Loading file: batch_dna_30.pkl
Loading file: batch_dna_31.pkl
Loading file: batc

{'val_seen_loc': <multimodal_dataset.MultiModalDataset at 0x7f440cd99910>,
 'val_unseen_loc': <multimodal_dataset.MultiModalDataset at 0x7f440ce31e90>,
 'test_seen_loc': <multimodal_dataset.MultiModalDataset at 0x7f440c353690>,
 'test_unseen_loc': <multimodal_dataset.MultiModalDataset at 0x7f43f5f9e110>}

In [55]:
device

device(type='cuda')

In [ ]:
# Evaluate:
from models import AttentionFusion, GenusClassifier, LocalSpecieClassfier, MainClassifier, multimodal_collector

fusion_embedder = AttentionFusion(dna_dim=512,img_dim=768,dna_len_dim=32, fused_dim=258, proj_dna_dim=128-32, proj_img_dim=128, dropout=0.2)
print("Fusion model created. fused dim: ", fusion_embedder.fused_dim)
genus_classifier = GenusClassifier(fusion_embedder.fused_dim,dropout=0.1, dna_len_dim=32)

local_specie_classifier = LocalSpecieClassfier(fusion_embedder.fused_dim,reduced_fused_dim=128, specie_decoder_hidden_dim=256, dropout=0.1,dna_len_dim=32)

main_classifier = MainClassifier(mat['species2genus'], genus_species, None, None, fusion_embedder, genus_classifier,local_specie_classifier,alpha=2, beta=0, theta=0,).to(device)



Fusion model created. fused dim:  258


MainClassifier(
  (fusion_embedder): AttentionFusion(
    (dna_len_emb): Embedding(120, 32)
    (proj_dna): Linear(in_features=512, out_features=96, bias=True)
    (proj_img): Linear(in_features=768, out_features=128, bias=True)
    (weight_dna_len): Linear(in_features=32, out_features=1, bias=False)
    (weight_dna): Linear(in_features=512, out_features=1, bias=False)
    (weight_img): Linear(in_features=768, out_features=1, bias=False)
    (ffn): Sequential(
      (0): Linear(in_features=256, out_features=258, bias=True)
    )
  )
  (genus_classifier): GenusClassifier(
    (decoder): Sequential(
      (0): Linear(in_features=290, out_features=744, bias=True)
      (1): Sigmoid()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=744, out_features=372, bias=True)
    )
  )
  (local_specie_classifier): LocalSpecieClassfier(
    (fused_proj): Linear(in_features=258, out_features=128, bias=True)
    (genus_embedder): Linear(in_features=372, out_features=64, bias=True)